<H1 style="text-align: center;">Hypoglycemia Detection</H1>
<H3 style="text-align: center;">Machine Learning</H3>
###Imports

In [489]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
import seaborn as sns
import copy
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier

In [506]:
df = pd.DataFrame([]) # defining a dataframe
df = pd.read_csv('../Data/Cleaned/data.csv')

df['Code'] = df['Code'].astype('category')

df['Date_Time'] = pd.to_datetime(df['Date_Time'].str.strip(), 
                                 format="%Y-%m-%d %H:%M:%S", errors="raise")

df['Time'] = df['Date_Time'].dt.time

df.head()

,Code,Blood_Glucose,Patient_ID,Code_Description,Date_Time,Time
0,58,101.0,31,PRE_BREAKFAST,1991-05-20 08:00:00,08:00:00
1,33,5.0,31,REGULAR_INSULIN_DOSE,1991-05-20 12:00:00,12:00:00
2,34,27.0,31,NPH_INSULIN_DOSE,1991-05-20 08:00:00,08:00:00
3,60,89.0,31,PRE_LUNCH,1991-05-20 12:00:00,12:00:00
4,33,3.0,31,REGULAR_INSULIN_DOSE,1991-05-20 12:00:00,12:00:00


In [507]:
df_agg=pd.DataFrame()

for i in range(1, 70):
    x = df.loc[df['Patient_ID']==i,['Blood_Glucose','Date_Time']].set_index('Date_Time').resample('240T').mean().reset_index()
    x['Time'] = x['Date_Time'].dt.time
    x = x.groupby('Time').agg(np.std).reset_index()
    x['Patient_ID'] = i
    x['Hypo_Flag'] = df.loc[(df['Patient_ID']==i) & (df['Code_Description']=='HYPOGLYCEMIC_SYMPTOMS')].any().count()>0
    df_agg = df_agg.append(x)

In [508]:
for i in range(1, 70):
    df_agg.loc[df_agg['Patient_ID']==i] = df_agg.loc[df_agg['Patient_ID']==i].fillna(df.loc[(df['Patient_ID']==i),'Blood_Glucose'].mean())
    
df_agg.head()

,Time,Blood_Glucose,Patient_ID,Hypo_Flag
0,00:00:00,67.783333,1,True
1,04:00:00,38.121757,1,True
2,08:00:00,67.783333,1,True
3,12:00:00,19.563280,1,True
4,16:00:00,65.874422,1,True


In [509]:
df_pivot = pd.pivot_table(df_agg, values='Blood_Glucose', index='Patient_ID',
                     columns=['Time'])

In [510]:
df_pivot.head()

Time,00:00:00,04:00:00,08:00:00,12:00:00,16:00:00,20:00:00
Patient_ID,,,,,,
1,67.783333,38.121757,67.783333,19.563280,65.874422,71.848756
2,100.045205,100.045205,63.285022,20.102470,75.986572,75.114214
3,70.313793,28.457325,23.522843,70.313793,55.974488,47.610959
4,82.479310,71.566297,24.863708,82.479310,38.969463,59.927845
5,85.882759,64.724393,22.641079,85.882759,67.988387,54.814814


In [511]:
df_pivot['Hypo_Flag'] = df.groupby('Patient_ID').agg(lambda row : (row['Code_Description']=='HYPOGLYCEMIC_SYMPTOMS').any()).any(axis=1)

In [512]:
df_pivot.head()

Time,00:00:00,04:00:00,08:00:00,12:00:00,16:00:00,20:00:00,Hypo_Flag
Patient_ID,,,,,,,
1,67.783333,38.121757,67.783333,19.563280,65.874422,71.848756,True
2,100.045205,100.045205,63.285022,20.102470,75.986572,75.114214,False
3,70.313793,28.457325,23.522843,70.313793,55.974488,47.610959,True
4,82.479310,71.566297,24.863708,82.479310,38.969463,59.927845,True
5,85.882759,64.724393,22.641079,85.882759,67.988387,54.814814,False


Best score is 0.7291666666666666


In [301]:
scaler = StandardScaler()
logreg = LogisticRegression()
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space, 'penalty': ['l1', 'l2']}

# Instantiate the GridSearchCV object: logreg_cv
logreg_cv = GridSearchCV(logreg, param_grid, cv=5 )

# Fit it to the training data
logreg_cv.fit(X_train, y_train)

# Print the optimal parameters and best score
print("Tuned Logistic Regression Parameter: {}".format(logreg_cv.best_params_))
print("Tuned Logistic Regression Accuracy: {}".format(logreg_cv.best_score_))

Tuned Logistic Regression Parameter: {'C': 3.7275937203149381, 'penalty': 'l1'}
Tuned Logistic Regression Accuracy: 0.6829268292682927


In [302]:
logreg_cv.predict(X_test)

logreg_cv.score(X_test, y_test)

0.5357142857142857

In [303]:
gCV.predict(X_test)

gCV.score(X_test, y_test)

0.5357142857142857

In [513]:
group_df = df.groupby('Patient_ID')

df_agg_cat = pd.DataFrame({
    
    
    'exercise_perday' : group_df.apply(
        lambda group : group[(group['Code_Description']=='TYPICAL_EXERCISE')]['Code_Description'].count()
        /((group.Date_Time.max() - group.Date_Time.min()).days)),
    'irregular_diet_perday' : group_df.apply(
        lambda group : group[(group['Code_Description']=='MORE_MEAL_INJESTION') |
                            (group['Code_Description']=='LESS_MEAL_INJESTION')]['Code_Description'].count()
        /((group.Date_Time.max() - group.Date_Time.min()).days)),
    'irregular_exercise_perday' : group_df.apply(
        lambda group : group[(group['Code_Description']=='LESS_EXERCISE') |
                            (group['Code_Description']=='MORE_EXERCISE')]['Code_Description'].count()
        /((group.Date_Time.max() - group.Date_Time.min()).days)),
    
    'snacks_perday' : group_df.apply(lambda group : group[group['Code_Description']=='PRE_SNACK']['Code_Description'].count()
                                    /(group.Date_Time.max() - group.Date_Time.min()).days),
    'nph_insulin_perday' : group_df.apply(lambda group : group[group['Code_Description']=='NPH_INSULIN_DOSE']['Code_Description'].count()
                                    /(group.Date_Time.max() - group.Date_Time.min()).days),
    'ultralente_insulin_perday' : group_df.apply(lambda group : group[group['Code_Description']=='ULTRALENTE_INSULIN_DOSE']['Code_Description'].count()
                                    /(group.Date_Time.max() - group.Date_Time.min()).days),
    'healthy_glucose_levels_perday' : group_df.apply(lambda group : group[(group['Blood_Glucose']>=80) & (group['Blood_Glucose']<=180)]['Code_Description'].count()
                                                     /(group.Date_Time.max() - group.Date_Time.min()).days),
    'mean' : group_df['Blood_Glucose'].mean(),
    'irregular_perday' : group_df.apply(
        lambda group : group[(group['Code_Description']=='LESS_EXERCISE') |
                            (group['Code_Description']=='MORE_EXERCISE') |
                            (group['Code_Description']=='MORE_MEAL_INJESTION') |
                            (group['Code_Description']=='LESS_MEAL_INJESTION')]['Code_Description'].count()
        /((group.Date_Time.max() - group.Date_Time.min()).days))
})

#df_agg2.loc[df_agg['postmeal_median_bg'].isnull(), 'postmeal_median_bg'] = df_agg.loc[df_agg['postmeal_median_bg'].isnull(), 'median']

df_agg_cat.head()

,exercise_perday,healthy_glucose_levels_perday,irregular_diet_perday,irregular_exercise_perday,irregular_perday,mean,nph_insulin_perday,snacks_perday,ultralente_insulin_perday
Patient_ID,,,,,,,,,
1,0.0,1.303704,0.000000,0.0,0.000000,67.783333,0.970370,0.000000,0.000000
2,0.0,1.421053,0.000000,0.0,0.000000,100.045205,0.021053,0.000000,0.000000
3,0.0,2.078947,0.263158,0.0,0.263158,70.313793,0.026316,0.394737,1.842105
4,0.0,1.552632,0.157895,0.0,0.157895,82.479310,1.421053,0.842105,0.473684
5,0.0,1.650000,0.050000,0.0,0.050000,85.882759,1.875000,0.900000,0.000000


In [514]:
df_pivot = df_pivot.join(df_agg_cat, how='inner')

In [515]:
df_pivot.head()

,00:00:00,04:00:00,08:00:00,12:00:00,16:00:00,20:00:00,Hypo_Flag,exercise_perday,healthy_glucose_levels_perday,irregular_diet_perday,irregular_exercise_perday,irregular_perday,mean,nph_insulin_perday,snacks_perday,ultralente_insulin_perday
Patient_ID,,,,,,,,,,,,,,,,
1,67.783333,38.121757,67.783333,19.563280,65.874422,71.848756,True,0.0,1.303704,0.000000,0.0,0.000000,67.783333,0.970370,0.000000,0.000000
2,100.045205,100.045205,63.285022,20.102470,75.986572,75.114214,False,0.0,1.421053,0.000000,0.0,0.000000,100.045205,0.021053,0.000000,0.000000
3,70.313793,28.457325,23.522843,70.313793,55.974488,47.610959,True,0.0,2.078947,0.263158,0.0,0.263158,70.313793,0.026316,0.394737,1.842105
4,82.479310,71.566297,24.863708,82.479310,38.969463,59.927845,True,0.0,1.552632,0.157895,0.0,0.157895,82.479310,1.421053,0.842105,0.473684
5,85.882759,64.724393,22.641079,85.882759,67.988387,54.814814,False,0.0,1.650000,0.050000,0.0,0.050000,85.882759,1.875000,0.900000,0.000000


In [550]:
data_array = df_pivot.values

Xs = np.array(df_pivot.values[:,[1,2,3,4,5,7,8,9,10,11,13,14,15]], dtype='f')
ys =  np.array(df_pivot.values[:,6], dtype='f')

X_train, X_test,y_train,  y_test = train_test_split(Xs,ys, test_size=0.4, random_state=20)



In [551]:
scaler = StandardScaler()
treeClassifier = DecisionTreeClassifier()

param_dist = {"max_depth": [5, None],
              "criterion": ["gini", "entropy"]}
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space, 'penalty': ['l1', 'l2']}

gCV = GridSearchCV(treeClassifier , {},cv=5 )
gCV.fit(X_train, y_train)

print("Best score is {}".format(gCV.best_score_))



Best score is 0.7804878048780488


In [552]:
scaler = StandardScaler()
logreg = LogisticRegression()
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space, 'penalty': ['l1', 'l2']}

# Instantiate the GridSearchCV object: logreg_cv
logreg_cv = GridSearchCV(logreg, {}, cv=5 )

# Fit it to the training data
logreg_cv.fit(X_train, y_train)

# Print the optimal parameters and best score
print("Tuned Logistic Regression Parameter: {}".format(logreg_cv.best_params_))
print("Tuned Logistic Regression Accuracy: {}".format(logreg_cv.best_score_))

Tuned Logistic Regression Parameter: {}
Tuned Logistic Regression Accuracy: 0.7560975609756098


In [553]:
logreg_cv.predict(X_test)

logreg_cv.score(X_test, y_test)

0.5714285714285714

In [554]:
gCV.predict(X_test)

gCV.score(X_test, y_test)

0.5357142857142857

In [347]:
np.logspace(-5, 8, 15)

array([  1.00000000e-05,   8.48342898e-05,   7.19685673e-04,
         6.10540230e-03,   5.17947468e-02,   4.39397056e-01,
         3.72759372e+00,   3.16227766e+01,   2.68269580e+02,
         2.27584593e+03,   1.93069773e+04,   1.63789371e+05,
         1.38949549e+06,   1.17876863e+07,   1.00000000e+08])

In [555]:
scaler = StandardScaler()
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, y_train)

knn.score(X_train, y_train)

0.68292682926829273

In [556]:
knn.predict(X_test)

knn.score(X_test, y_test)

0.42857142857142855

In [557]:
scaler = StandardScaler()
rfc = RandomForestClassifier()
rf_grid =GridSearchCV(rfc, {}, cv=5 )
rf_grid.fit(X_train, y_train)

rf_grid.score(X_train, y_train)

1.0

In [558]:
rf_grid.predict(X_test)

rf_grid.score(X_test, y_test)

0.5714285714285714